In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
data = pd.read_csv('E:/kaggle_games/Human_Activity_Recognition/train.csv')

In [3]:
data.head()

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",subject,Activity
0,0.288585,-0.020294,-0.132905,-0.995279,-0.983111,-0.913526,-0.995112,-0.983185,-0.923527,-0.934724,...,-0.710304,-0.112754,0.030400,-0.464761,-0.018446,-0.841247,0.179941,-0.058627,1,STANDING
1,0.278419,-0.016411,-0.123520,-0.998245,-0.975300,-0.960322,-0.998807,-0.974914,-0.957686,-0.943068,...,-0.861499,0.053477,-0.007435,-0.732626,0.703511,-0.844788,0.180289,-0.054317,1,STANDING
2,0.279653,-0.019467,-0.113462,-0.995380,-0.967187,-0.978944,-0.996520,-0.963668,-0.977469,-0.938692,...,-0.760104,-0.118559,0.177899,0.100699,0.808529,-0.848933,0.180637,-0.049118,1,STANDING
3,0.279174,-0.026201,-0.123283,-0.996091,-0.983403,-0.990675,-0.997099,-0.982750,-0.989302,-0.938692,...,-0.482845,-0.036788,-0.012892,0.640011,-0.485366,-0.848649,0.181935,-0.047663,1,STANDING
4,0.276629,-0.016570,-0.115362,-0.998139,-0.980817,-0.990482,-0.998321,-0.979672,-0.990441,-0.942469,...,-0.699205,0.123320,0.122542,0.693578,-0.615971,-0.847865,0.185151,-0.043892,1,STANDING


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7352 entries, 0 to 7351
Columns: 563 entries, tBodyAcc-mean()-X to Activity
dtypes: float64(561), int64(1), object(1)
memory usage: 31.6+ MB


In [5]:
sum(data.isnull().sum())

0

In [6]:
# 思路：维数太多，考虑降维（pca）；归一化；
#降维方法：
# 1、缺失值比率；2、低方差滤波（df.var()）；3、高相关滤波(df.corr())；4、随机森林（可得出特征重要性）；5、反向特征消除；
# 6、前向特征选择（5，6仅适用于特征较少的情况）；7、因子分析；8、主成分分析（pca）;9、独立分量分析（ICA）;10、LDA


In [7]:
# 1、缺失值比率
sum(data.isnull().sum()/len(data)*100)

0.0

In [8]:
# 2、低方差滤波
va = data.var()
variables = data.columns
data1 = data.copy()
variable = []
for i in range(len(va)):
    if va[i] > 0.1:
        variable.append(variables[i])

In [9]:
len(variable)

190

In [10]:
data2 = data1[variable]

In [11]:
data2.head()

,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,tBodyAcc-min()-X,tBodyAcc-min()-Y,tBodyAcc-sma(),...,fBodyBodyGyroMag-skewness(),fBodyBodyGyroMag-kurtosis(),fBodyBodyGyroJerkMag-entropy(),fBodyBodyGyroJerkMag-skewness(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)",subject
0,-0.995279,-0.983111,-0.913526,-0.995112,-0.983185,-0.923527,-0.934724,0.852947,0.685845,-0.965523,...,0.586156,0.374605,-0.871306,-0.298676,-0.112754,0.030400,-0.464761,-0.018446,-0.841247,1
1,-0.998245,-0.975300,-0.960322,-0.998807,-0.974914,-0.957686,-0.943068,0.849308,0.685845,-0.981930,...,-0.336310,-0.720015,-1.000000,-0.595051,0.053477,-0.007435,-0.732626,0.703511,-0.844788,1
2,-0.995380,-0.967187,-0.978944,-0.996520,-0.963668,-0.977469,-0.938692,0.843609,0.682401,-0.983478,...,-0.535352,-0.871914,-1.000000,-0.390748,-0.118559,0.177899,0.100699,0.808529,-0.848933,1
3,-0.996091,-0.983403,-0.990675,-0.997099,-0.982750,-0.989302,-0.938692,0.843609,0.682401,-0.986093,...,-0.230091,-0.511217,-0.955696,-0.117290,-0.036788,-0.012892,0.640011,-0.485366,-0.848649,1
4,-0.998139,-0.980817,-0.990482,-0.998321,-0.979672,-0.990441,-0.942469,0.849095,0.683250,-0.992653,...,-0.510282,-0.830702,-1.000000,-0.351471,0.123320,0.122542,0.693578,-0.615971,-0.847865,1


In [12]:
# 3、高相关滤波(df.corr())
data2.corr()

,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,tBodyAcc-min()-X,tBodyAcc-min()-Y,tBodyAcc-sma(),...,fBodyBodyGyroMag-skewness(),fBodyBodyGyroMag-kurtosis(),fBodyBodyGyroJerkMag-entropy(),fBodyBodyGyroJerkMag-skewness(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)",subject
tBodyAcc-std()-X,1.000000,0.927461,0.851668,0.998632,0.920888,0.846392,0.980844,-0.966500,-0.904539,0.973155,...,-0.246488,-0.244528,0.907663,0.148034,-0.035011,-0.021633,0.018985,-0.024810,-0.371653,-0.064345
tBodyAcc-std()-Y,0.927461,1.000000,0.895510,0.922803,0.997347,0.894509,0.917366,-0.937918,-0.957736,0.971500,...,-0.222345,-0.223777,0.941676,0.206227,-0.020379,-0.012505,-0.008507,-0.014592,-0.380531,-0.115524
tBodyAcc-std()-Z,0.851668,0.895510,1.000000,0.844469,0.891441,0.997418,0.853884,-0.860691,-0.853346,0.928042,...,-0.170854,-0.173792,0.907742,0.157792,-0.006769,-0.020036,-0.018429,-0.006471,-0.345011,-0.050123
tBodyAcc-mad()-X,0.998632,0.922803,0.844469,1.000000,0.916106,0.839267,0.973216,-0.962235,-0.900336,0.970683,...,-0.246336,-0.244121,0.900842,0.149257,-0.042713,-0.021537,0.019389,-0.024951,-0.368191,-0.063440
tBodyAcc-mad()-Y,0.920888,0.997347,0.891441,0.916106,1.000000,0.891178,0.910411,-0.933135,-0.941377,0.968444,...,-0.216630,-0.219116,0.935176,0.200890,-0.023722,-0.012310,-0.012546,-0.012341,-0.377025,-0.114753
tBodyAcc-mad()-Z,0.846392,0.894509,0.997418,0.839267,0.891178,1.000000,0.847870,-0.856964,-0.848485,0.926489,...,-0.163409,-0.167514,0.904920,0.157937,-0.008768,-0.020508,-0.023525,-0.007231,-0.347389,-0.055457
tBodyAcc-max()-X,0.980844,0.917366,0.853884,0.973216,0.910411,0.847870,1.000000,-0.941451,-0.898652,0.956887,...,-0.242664,-0.240810,0.909701,0.154220,-0.033048,-0.021895,0.025066,-0.028871,-0.384192,-0.055633
tBodyAcc-min()-X,-0.966500,-0.937918,-0.860691,-0.962235,-0.933135,-0.856964,-0.941451,1.000000,0.910726,-0.961854,...,0.252642,0.249789,-0.922069,-0.166157,0.011599,0.023030,-0.002334,0.017876,0.362847,0.085809
tBodyAcc-min()-Y,-0.904539,-0.957736,-0.853346,-0.900336,-0.941377,-0.848485,-0.898652,0.910726,1.000000,-0.930699,...,0.237443,0.234137,-0.911485,-0.207130,0.014729,0.014424,-0.010098,0.030691,0.367873,0.111776
tBodyAcc-sma(),0.973155,0.971500,0.928042,0.970683,0.968444,0.926489,0.956887,-0.961854,-0.930699,1.000000,...,-0.218643,-0.219757,0.944032,0.169745,-0.022561,-0.014693,-0.001252,-0.017063,-0.369237,-0.075566


In [13]:
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel

In [14]:
label = data['Activity']
feature = data.drop('Activity', axis=1)

In [15]:
lsvc = LinearSVC(C=0.001, penalty='l1', dual=False).fit(feature, label)
model = SelectFromModel(lsvc, prefit=True)
feature_new = model.transform(feature)

In [16]:
feature_new.shape

(7352, 23)

In [17]:
label.value_counts()

LAYING                1407
STANDING              1374
SITTING               1286
WALKING               1226
WALKING_UPSTAIRS      1073
WALKING_DOWNSTAIRS     986
Name: Activity, dtype: int64

In [18]:
from sklearn.preprocessing import LabelEncoder
enc = LabelEncoder()
label = enc.fit_transform(label)

In [19]:
label

array([2, 2, 2, ..., 5, 5, 5], dtype=int64)

In [20]:
from sklearn.preprocessing import MinMaxScaler
scale = MinMaxScaler()
feature = scale.fit_transform(data2)

In [21]:
from sklearn.linear_model import Lasso
lasso = Lasso(alpha=0.01)
lasso.fit(feature, label)

Lasso(alpha=0.01, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [22]:
mask = lasso.coef_ != 0
feature_new1 = feature[:,mask]
feature_new1.shape

(7352, 23)

In [23]:
label

array([2, 2, 2, ..., 5, 5, 5], dtype=int64)

In [24]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(feature_new, label, test_size=0.3, random_state=0)
X_train1, X_test1, y_train1, y_test1 = train_test_split(feature_new1, label, test_size=0.3, random_state=0)

In [25]:
# 建模
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
models = {
    'SVC':SVC(kernel='rbf'),
    'RandomForestClassifier':RandomForestClassifier(n_estimators=200),
    'GradientBoostingClassifier':GradientBoostingClassifier(n_estimators=200)
}

D:\Anaconda\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [26]:
def train_eval_model(models, X_train, X_test, y_train, y_test):
    for name, model in models.items():
        print(name,':')
        model.fit(X_train, y_train)
        y_predict = model.predict(X_test)
        print('accuracy:%.3f'%accuracy_score(y_test, y_predict))
        #print('confusion_matrix:\n', confusion_matrix(y_test, y_predict))

train_eval_model(models, X_train, X_test, y_train, y_test)

SVC :
accuracy:0.927
RandomForestClassifier :
accuracy:0.968
GradientBoostingClassifier :
accuracy:0.979


In [27]:
train_eval_model(models, X_train1, X_test1, y_train1, y_test1)

SVC :
accuracy:0.893
RandomForestClassifier :
accuracy:0.972
GradientBoostingClassifier :
accuracy:0.972
